# Práctico Content-based (Texto)

**Profesor:** Denis Parra

**Alumno:** `ESCRIBIR TU NOMBRE AQUI`


En este proyecto trabajaremos con un modelo de recomendacion de libros de la página [Goodreads](http://www.goodreads.com). El modelo de recomendación de libros es un recomendador basado en contenido, donde se utilizan modelos de lenguage BERT y BERT-large para el cálculo de embeddings de los libros y luego similaridades de ítems. Luego, dependiendo de los libros con los que el usuario ha interactuado, se recomiendan los ítems más similares.

In [ ]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time

Descargamos datos que vienen previamente calculados:
- transacciones/interacciones de cada usuario
- transaciones para evaluar el modelo
- embeddings de descripciones calculados con BERT  
- embeddings de descripciones calculados con BERT-large
- datos de libros con información de titulo, descripcion, año de publicacion, entre otros.

In [ ]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

# Cargar datos adicionales

In [ ]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()

In [ ]:
df_books.columns

In [ ]:
print("# ítems   --> ", df_books["book_id"].nunique())

In [ ]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)


In [ ]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

# Cargar características pre-entrenadas: BERT y BERT-large

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT y BERT-large que convierten texto a embeddings.

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin.

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador.

Tiene dos versiones:
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros. Genera vectores de 1024 dimensiones

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

En este caso los textos que utilizaremos son los títulos de los libros con su descripción y compararemos los resultados de recomendación con BERT y BERT-large. Para efectos de este trabajo los vectores de características ya fueron entrenados y guardados en archivos numpy. A continuación son cargados en memoria.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)


In [ ]:
bert_featmat = np.load('goodreads_bert_embeddings.npy', allow_pickle=True)
bert_large_featmat = np.load('goodreads_bert_large_embeddings.npy', allow_pickle=True)

In [ ]:
bert_featmat.shape

In [ ]:
bert_large_featmat.shape

### **Pregunta 1**

Considerando que haremos un recomendador basado en contenidos ¿Por qué el uso de modelos de lenguage es una buena elección para este tipo de problema?

**Respuesta 1:**

# Probamos con BERT y BERT-large reduciendo dimensionalidad con PCA-20

Una vez calculado (o cargado) los vectores característicos de cada libro a partir de su titulo y descripción, reducimos dimensionalidad. Probaremos con BERT y BERT-large para comparar los resultados de ambos en recomendación basada en contenido.


In [ ]:
# Project into a 20 PCA feature space
pca20_bert_featmat = PCA(n_components=20).fit_transform(bert_featmat)
pca20_bert_large_featmat = PCA(n_components=20).fit_transform(bert_large_featmat)

In [ ]:
pca20_bert_featmat.shape

In [ ]:
pca20_bert_large_featmat.shape

### **Pregunta 2**

Comente por qué se utiliza PCA para reducir la dimensión de cada vector característico. ¿Qué sucede con la pérdida de información en la reducción de dimensionalidad?

**Respuesta 2:**

# Similar document retrieval

En esta sección utilizaremos los vectores cargados para hacer un sistema de recuperación o búsqueda de información, para diferentes métricas de distancia.

Buscamos libros similares de acuerdo a la representación vectorial (BERT) de su título y descripción.


In [ ]:
# format results
pd.options.display.max_colwidth = 50 # ...máximo de columna

In [ ]:
# Find similar images by image id
def find_similar_books(embedding, query_id=None, metric='euclidean', topk=10):

    n = embedding.shape[0]

    if query_id is None:
        query_i = np.random.randint(n)
        query_id = idx2bookid[query_i]

    else:
        query_i = bookid2idx[query_id]


    distances = pairwise_distances(embedding[query_i].reshape(1,-1), embedding, metric=metric)
    heap = []
    for i in range(n):
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[0][i], i))
        else:
            heapq.heappushpop(heap, (-distances[0][i], i))

    heap.sort(reverse=True)
    rec_ids = [idx2bookid[i] for _,i in heap]

    return rec_ids

## Usando BERT

In [ ]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine"
similar_books = find_similar_books(bert_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

## Usando BERT reducidos con PCA

In [ ]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine"
similar_books = find_similar_books(pca20_bert_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

## Usando BERT-large

In [ ]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine"
similar_books = find_similar_books(bert_large_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

## Usando BERT-large reducidos con PCA

In [ ]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine"
similar_books = find_similar_books(pca20_bert_large_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

### Pregunta 3:
Comente los resultados obtenidos, en cuanto a modelo de lenguaje, reduccion de dimensionalidad y métrica de distancia utilizada.


**Respuesta 3:**

# Recomendaciones

In [ ]:
# format results
pd.options.display.max_colwidth = 50
#pd.set_option('display.max_colwidth', -1)

## Función para obtener recomendacion para cada usuario

In [ ]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):

    #print("user_id = ", user_id)

    user_id = str(user_id)

    #Calculate distance metrics
    trx = user_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9

    # recorremos transacciones pasadas del usuario
    for t in trx:
        query_i = bookid2idx[t]

        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []
    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))
    heap.sort(reverse=True)

    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia
    recommended_ids = [idx2bookid[i] for _,i in heap]

    # retornar los que el usuario no haya consumido
    filtered_recommended_ids = []

    return recommended_ids

## Generar recomendaciones para un usuario en particular

In [ ]:
# recomendación para el usuario id = 49299 , utilizando bert con reduccion de dimensionalidad a 20
user_id = '50101'
rec = recommend(pca20_bert_featmat, user_id=user_id, topk=15)
rec

## Transacciones pasadas del usuario

In [ ]:
past_interactions = user_interactions[str(user_id)]

df_books[df_books['book_id'].isin(past_interactions)][['book_id', 'original_title', 'book_desc', 'authors']]


## Información de recomendaciones

In [ ]:
df_books[df_books['book_id'].isin(rec)][['book_id', 'original_title', 'book_desc', 'authors']]

# Evaluación de las recomendaciones con interacciones de testing

In [ ]:
# Métricas de evaluación
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg


## Evaluación de recomendación con BERT

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))


## Evaluación de recomendación con BERT reducidos con PCA-20

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

## Evaluación de recomendación con BERT-large

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

## Evaluación de recomendación con BERT-large reducidos con PCA-20

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

topk = 10

for i, u in enumerate(user_interactions_test.keys()):

    print(i, end= '\r')

    rec = recommend(pca20_bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start


In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

### Pregunta 4:
- Comente los resultados en términos de tiempo de ejecución y métricas de ranking para los 4 modelos.

**Respuesta 4:**